In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
def extract_number(item,nth):
    res = int(re.findall(r'\d+', str(item))[nth])
    return res

In [3]:
def get_primary_info(info):
    record = []
    try:
        content = info.contents[1]            
        # get id
        links = content.get("href")
        REid = re.findall(r'\d+',links.split("-")[-1])[0]
        record.append(REid)
        # get url
        url = "http://www.century21.com" + links
        record.append(url)
        # get price
        price = int(''.join(re.findall(r'\d+', content.text)))
        record.append(price)
    except:
        record.append('NA')
            
    try:
        # get bedroomcnt
        beds = extract_number(info.find_all("div",{"class":"property-beds"})[0].contents[1],0)
        record.append(beds)
    except:
        record.append('NA')
        
    try:
        # get bathroomcnt
        baths = extract_number(info.find_all("div",{"class":"property-baths"})[0].contents[1],0)
        record.append(baths)
    except:
        record.append('NA')
        
    try:
        # get half bathroomcnt
        hbaths = extract_number(info.find_all("div",{"class":"property-half-baths"})[0].contents[1],0)
        record.append(hbaths)
    except:
        record.append('NA')
       
    try:
        # get sqft
        sqft =''.join(re.findall(r'\d+', str(info.find_all("div",{"class":"property-sqft"})[0].contents[1])))
        record.append(sqft)
    except:
        record.append('NA')
         
    try:
        # get address
        address = info.find_all("div",{"class":"property-address"})[0].text.strip()
        record.append(address)
    except:
        data_file.write("|")
            
    try:
        # get zipcode
        zipcode = re.findall(r'\d+', str(info.find_all("div",{"class":"property-city"})[0].text))[0]
        record.append(zipcode)
    except:
        record.append('NA')    
            
    try:
        # get card attribution
        attribution = info.find_all("div",{"class":"property-card-attribution"})[0].text.strip()
        record.append(attribution)
    except:
        record.append('NA')


    return record
    

In [4]:
url = "http://www.century21.com/real-estate/arlington-va/LCVAARLINGTON/"
page =requests.get(url)
contents=page.content
soup=BeautifulSoup(contents)

/Users/Shuyuan/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/Shuyuan/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [5]:
primary_info=soup.find_all("div",{"class":"property-card-primary-info"}) 
records = []
columns = ['REid','rul','price','beds','baths','hbaths','sqft','address','zipcode','attribution']
df = pd.DataFrame(columns=columns)

In [6]:
i = 0
for info in primary_info:
    df.loc[i] = get_primary_info(info)
    i += 1
    records.append(get_primary_info(info))


In [7]:
df

,REid,rul,price,beds,baths,hbaths,sqft,address,zipcode,attribution
0,030968273,http://www.century21.com/property/601-frederic...,948000,4,4,1,3666,601 Frederick Street North,22203,Courtesy Of RE/MAX Allegiance
1,030966895,http://www.century21.com/property/1501-inglewo...,915000,3,2,1,2538,1501 Inglewood Street,22205,Courtesy Of Washington Fine Properties LLC
2,030967844,http://www.century21.com/property/1418-queen-s...,779000,4,3,NA,1785,1418 Queen Street,22204,"Courtesy Of RE/MAX Distinctive Real Estate, Inc."
3,030965829,http://www.century21.com/property/1315-queen-s...,669900,3,2,1,1570,1315 Queen Street South,22204,Courtesy Of Long & Foster Real Estate
4,030966796,http://www.century21.com/property/117-buchanan...,649000,3,2,NA,1644,117 Buchanan Street South,22204,"Courtesy Of Weichert, REALTORS"
5,030966260,http://www.century21.com/property/2016-adams-s...,344900,2,1,NA,938,2016 Adams Street North 106,22201,Courtesy Of Jack Lawlor Realty Company
6,030965348,http://www.century21.com/property/1808-queens-...,255000,1,1,NA,604,1808 Queens Lane 213,22201,Courtesy Of Keller Williams Realty
7,030951587,http://www.century21.com/property/2825-arizona...,1849900,5,5,1,NA,2825 Arizona Street North,22213,"Courtesy Of Yeonas & Shafran Real Estate, LLC"
8,030954169,http://www.century21.com/property/3800-lee-hig...,515000,2,2,NA,1036,3800 Lee Highway 205,22207,Courtesy Of Fairfax Realty
9,2180365795,http://www.century21.com/property/2220-fairfax...,499000,1,1,1,887,2220 Fairfax Dr 705,22201,Listed By CENTURY 21 Redwood Realty
